In [6]:
import fastplotlib as fpl
import masknmf
import tifffile
import torch
import numpy as np
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load dataset of interest

In [2]:
data = tifffile.imread("/path/to/data.tiff")

# Run PMD decomposition. Result is a PMDArray object, which behaves like an array (supports slicing, etc.) and also supports "to" (so you can move it back and forth from cuda)

In [13]:
block_sizes = [32, 32]
max_components = 20
background_rank = 2
device = "cuda"
pixel_weighting = np.std(data, axis = 0) #This is an optional parameter
num_frames_for_spatial_fit = data.shape[0] #How many frames we use to estimate the spatial basis in PMD

pmd_result = masknmf.compression.pmd_decomposition(data,
                                                block_sizes,
                                                num_frames_for_spatial_fit,
                                                max_components = max_components,
                                                background_rank = background_rank,
                                               device = device,
                                               pixel_weighting=pixel_weighting,
                                               frame_batch_size = 1024)

# Move PMD to or from cuda

In [10]:
pmd_result.to('cuda')

# Use fastplotlib to see the raw and PMD results side by side. Note that even the PMDArray is on CUDA, calls to __getitem__ return numpy arrays (on CPU):

In [12]:
iw = fpl.ImageWidget(data = [data, pmd_result])
iw.cmap = "gray"
iw.show()